In [11]:
from scapy.all import *
from scapy.layers.inet import IP, TCP, UDP
from scapy.layers.inet6 import IPv6
import socket, struct

packets = rdpcap(r"C:\Users\jaysu\OneDrive\Desktop\COLLEGE\cn\lab6.pcapng")

def ones_complement_sum(words):
    total = 0
    for w in words:
        total += w
        total = (total & 0xFFFF) + (total >> 16)
    return total

def finalize_checksum(total):
    return (~total) & 0xFFFF

print("Showing checksum calculations (first 40 packets)...\n")

for i, pkt in enumerate(packets[:40], 1):
    print(f"\n===== Packet {i} =====")

    if IP in pkt:
        print("IPv4 packet detected")

    if IPv6 in pkt:
        ip6 = pkt[IPv6]
        print("IPv6 packet detected")

        src = socket.inet_pton(socket.AF_INET6, ip6.src)
        dst = socket.inet_pton(socket.AF_INET6, ip6.dst)

        if TCP in pkt:
            tcp = pkt[TCP]
            original = tcp.chksum
            segment = bytes(tcp)

            if len(segment) % 2:
                segment += b"\x00"

            pseudo = src + dst + struct.pack("!I3xB", len(segment), socket.IPPROTO_TCP)

            words = []
            for j in range(0, len(pseudo), 2):
                words.append((pseudo[j] << 8) + pseudo[j+1])
            for j in range(0, len(segment), 2):
                if j == 16:
                    words.append(0)
                else:
                    words.append((segment[j] << 8) + segment[j+1])

            calc = finalize_checksum(ones_complement_sum(words))
            print("TCP Calculated:", hex(calc))
            print("TCP Original  :", hex(original))

        if UDP in pkt:
            udp = pkt[UDP]
            original = udp.chksum
            segment = bytes(udp)

            if len(segment) % 2:
                segment += b"\x00"

            pseudo = src + dst + struct.pack("!I3xB", len(segment), socket.IPPROTO_UDP)

            words = []
            for j in range(0, len(pseudo), 2):
                words.append((pseudo[j] << 8) + pseudo[j+1])
            for j in range(0, len(segment), 2):
                if j == 6:
                    words.append(0)
                else:
                    words.append((segment[j] << 8) + segment[j+1])

            calc = finalize_checksum(ones_complement_sum(words))
            print("UDP Calculated:", hex(calc))
            print("UDP Original  :", hex(original))


Showing checksum calculations (first 40 packets)...


===== Packet 1 =====
IPv6 packet detected

===== Packet 2 =====
IPv6 packet detected

===== Packet 3 =====

===== Packet 4 =====

===== Packet 5 =====
IPv6 packet detected
UDP Calculated: 0x3900
UDP Original  : 0xd2b3

===== Packet 6 =====
IPv6 packet detected
UDP Calculated: 0xf001
UDP Original  : 0xf001

===== Packet 7 =====
IPv6 packet detected
UDP Calculated: 0x2429
UDP Original  : 0xce67

===== Packet 8 =====
IPv6 packet detected
UDP Calculated: 0x80e
UDP Original  : 0x80e

===== Packet 9 =====
IPv6 packet detected
UDP Calculated: 0xa4e8
UDP Original  : 0xa4e9

===== Packet 10 =====
IPv6 packet detected
UDP Calculated: 0xa267
UDP Original  : 0xce6a

===== Packet 11 =====
IPv6 packet detected
UDP Calculated: 0x9863
UDP Original  : 0x9863

===== Packet 12 =====
IPv6 packet detected
UDP Calculated: 0x5a3e
UDP Original  : 0x10a3

===== Packet 13 =====
IPv6 packet detected
UDP Calculated: 0x4e7a
UDP Original  : 0x10a9

===== Packet 